In [106]:
import fastf1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras import layers

In [107]:
def get_driver_data(driver):
    session = fastf1.get_session(2024, 'Silverstone', 'R')
    session.load(telemetry=True) 

    driver_general_data = pd.DataFrame(session.laps.pick_driver(driver))

    driver_data = session.laps.pick_driver(driver)
    return driver_data
    

In [108]:
def adjust_laptimes(dataframe):
    dataframe["Laptime"] = dataframe["LapTime"].astype(int) * 1e-9  # Converts LapTime to seconds
    dataframe["Position(s)"] = dataframe["Position"].astype(int)  # Converts Position to integer
    new_df = dataframe.drop(columns=["LapTime", "Position"])  # Drops LapTime and Position columns
    new_df = new_df.rename(columns={"Position(s)": "Position"})  # Renames Position(s) to Position
    return new_df

def get_common(dataframe):
    new_df = dataframe[["LapNumber", "TrackStatus"]]  # Selects only LapNumber and TrackStatus
    return new_df

def get_pitstop(dataframe):
    # Replace NaT or NaN with 0 for both columns
    dataframe["PitInTime"] = dataframe["PitInTime"].fillna(pd.Timedelta(0))
    dataframe["PitOutTime"] = dataframe["PitOutTime"].fillna(pd.Timedelta(0))
    
    # Convert Timedelta to seconds using total_seconds()
    dataframe["PitInTime"] = dataframe["PitInTime"].apply(lambda x: x.total_seconds() if isinstance(x, pd.Timedelta) else x)
    dataframe["PitOutTime"] = dataframe["PitOutTime"].apply(lambda x: x.total_seconds() if isinstance(x, pd.Timedelta) else x)
    
    # Create the PitstopTime column as the difference between PitOutTime and PitInTime
    dataframe["PitstopTime"] = dataframe["PitOutTime"] - dataframe["PitInTime"]
    
    return dataframe


# def adjust_pitstop_times(dataframe):
    

In [109]:
def remove_extras(dataframe):
    new_df = dataframe[["Driver", "Laptime", "PitstopTime", "Position", "Compound", "Position", "Source"]]  # Drops unnecessary columns
    return new_df


In [110]:
def drop_unecessary_telem(dataframe):
    dropped_dataframe = dataframe.drop(columns=["DriverAhead", 'SessionTime', 'Date', 'Time'])
    guess_free_dataframe = dropped_dataframe[dropped_dataframe['Source'] != 'interpolation']
    return guess_free_dataframe

In [111]:
driver_telemetry = pd.DataFrame()

bot_data = get_driver_data('BOT')


core           INFO 	Loading data for British Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info


req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '1', '4', '81', '55', '27', '18', '14', '23', '22', '2', '20', '3', '16', '77', '31', '11', '24', '63', '10']
c:\Users\smmey\OneDrive\Documents\f1_lap_by_lap\f1env\Lib\site-packages\fastf1\core.py:3067: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers

In [112]:
bot_data

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
726,0 days 01:00:00.034000,BOT,77,0 days 00:01:44.290000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:40.380000,...,True,Kick Sauber,0 days 00:58:15.512000,2024-07-07 14:03:12.541,1,19.0,False,,False,False
727,0 days 01:01:35.402000,BOT,77,0 days 00:01:35.368000,2.0,1.0,NaT,NaT,0 days 00:00:30.060000,0 days 00:00:39.194000,...,True,Kick Sauber,0 days 01:00:00.034000,2024-07-07 14:04:57.063,1,18.0,False,,False,True
728,0 days 01:03:09.849000,BOT,77,0 days 00:01:34.447000,3.0,1.0,NaT,NaT,0 days 00:00:30.096000,0 days 00:00:38.414000,...,True,Kick Sauber,0 days 01:01:35.402000,2024-07-07 14:06:32.431,1,18.0,False,,False,True
729,0 days 01:04:44.171000,BOT,77,0 days 00:01:34.322000,4.0,1.0,NaT,NaT,0 days 00:00:29.890000,0 days 00:00:38.839000,...,True,Kick Sauber,0 days 01:03:09.849000,2024-07-07 14:08:06.878,1,19.0,False,,False,True
730,0 days 01:06:17.939000,BOT,77,0 days 00:01:33.768000,5.0,1.0,NaT,NaT,0 days 00:00:29.920000,0 days 00:00:38.115000,...,True,Kick Sauber,0 days 01:04:44.171000,2024-07-07 14:09:41.200,1,19.0,False,,False,True
731,0 days 01:07:51.399000,BOT,77,0 days 00:01:33.460000,6.0,1.0,NaT,NaT,0 days 00:00:29.755000,0 days 00:00:37.957000,...,True,Kick Sauber,0 days 01:06:17.939000,2024-07-07 14:11:14.968,1,19.0,False,,False,True
732,0 days 01:09:25.047000,BOT,77,0 days 00:01:33.648000,7.0,1.0,NaT,NaT,0 days 00:00:29.919000,0 days 00:00:37.898000,...,True,Kick Sauber,0 days 01:07:51.399000,2024-07-07 14:12:48.428,1,19.0,False,,False,True
733,0 days 01:10:58.608000,BOT,77,0 days 00:01:33.561000,8.0,1.0,NaT,NaT,0 days 00:00:29.899000,0 days 00:00:37.968000,...,True,Kick Sauber,0 days 01:09:25.047000,2024-07-07 14:14:22.076,1,19.0,False,,False,True
734,0 days 01:12:32.253000,BOT,77,0 days 00:01:33.645000,9.0,1.0,NaT,NaT,0 days 00:00:29.917000,0 days 00:00:37.913000,...,True,Kick Sauber,0 days 01:10:58.608000,2024-07-07 14:15:55.637,1,19.0,False,,False,True
735,0 days 01:14:06.040000,BOT,77,0 days 00:01:33.787000,10.0,1.0,NaT,NaT,0 days 00:00:29.850000,0 days 00:00:38.666000,...,True,Kick Sauber,0 days 01:12:32.253000,2024-07-07 14:17:29.282,1,18.0,False,,False,True


In [113]:
for lap_number, lap in bot_data.iterlaps():
    telemetry = lap.get_telemetry()
    telemetry[['LapNumber', 'Laptime']] = lap[['LapNumber', 'LapTime']]

    driver_telemetry = pd.concat([driver_telemetry, telemetry])

driver_telemetry

C:\Users\smmey\AppData\Local\Temp\ipykernel_41876\2697744677.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  telemetry[['LapNumber', 'Laptime']] = lap[['LapNumber', 'LapTime']]
C:\Users\smmey\AppData\Local\Temp\ipykernel_41876\2697744677.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  telemetry[['LapNumber', 'Laptime']] = lap[['LapNumber', 'LapTime']]
C:\Users\smmey\AppData\Local\Temp\ipykernel_41876\2697744677.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a va

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,LapNumber,Laptime
2,2024-07-07 14:03:12.541,0 days 00:58:15.512000,,0.088889,0 days 00:00:00,10545.800004,0.000000,1,23.425000,False,1,interpolation,0.019699,0.000003,OnTrack,-1768.914203,1192.129451,1963.000018,1.0,0 days 00:01:44.290000
3,2024-07-07 14:03:12.660,0 days 00:58:15.631000,,0.088889,0 days 00:00:00.119000,10539.000000,0.000000,1,23.000000,False,1,car,0.000000,0.000000,OnTrack,-1768.894531,1192.159145,1963.000023,1.0,0 days 00:01:44.290000
4,2024-07-07 14:03:12.831,0 days 00:58:15.802000,,0.088889,0 days 00:00:00.290000,10126.996721,0.000000,1,23.534375,False,1,pos,-0.069833,-0.000012,OnTrack,-1769.000000,1192.000000,1963.000000,1.0,0 days 00:01:44.290000
5,2024-07-07 14:03:12.931,0 days 00:58:15.902000,,0.088889,0 days 00:00:00.390000,9886.059221,0.000000,1,23.846875,False,1,pos,-0.052149,-0.000009,OnTrack,-1769.000000,1192.000000,1963.000000,1.0,0 days 00:01:44.290000
6,2024-07-07 14:03:12.980,0 days 00:58:15.951000,14,0.088889,0 days 00:00:00.439000,9768.000000,0.000000,1,24.000000,False,1,car,0.000000,0.000000,OnTrack,-1768.692369,1192.464191,1963.000066,1.0,0 days 00:01:44.290000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,2024-07-07 15:26:21.236,0 days 02:21:24.207000,20,2481.143333,0 days 00:01:31.800000,11434.000000,248.000000,6,100.000000,False,0,car,5788.462500,0.994711,OnTrack,-1937.069258,961.733418,1962.531494,51.0,0 days 00:01:32.217000
707,2024-07-07 15:26:21.333,0 days 02:21:24.304000,20,2481.224167,0 days 00:01:31.897000,11490.467820,248.692857,6,100.000000,False,0,pos,5795.173175,0.995864,OnTrack,-1894.000000,1022.000000,1963.000000,51.0,0 days 00:01:32.217000
708,2024-07-07 15:26:21.516,0 days 02:21:24.487000,20,2481.376667,0 days 00:01:32.080000,11597.000000,250.000000,6,100.000000,False,0,car,5807.906944,0.998052,OnTrack,-1816.087263,1128.527684,1963.000556,51.0,0 days 00:01:32.217000
709,2024-07-07 15:26:21.573,0 days 02:21:24.544000,20,2481.376667,0 days 00:01:32.137000,11297.750336,251.781248,6,100.000000,False,0,pos,5811.922395,0.998742,OnTrack,-1792.000000,1161.000000,1963.000000,51.0,0 days 00:01:32.217000


In [114]:
driver_telemetry = pd.DataFrame(drop_unecessary_telem(driver_telemetry))

In [115]:
def adjust_laptimes(dataframe: pd.DataFrame):
    dataframe["Laptime"] = dataframe["Laptime"].astype("int64") * 1e-9  # Converts LapTime to seconds # Drops LapTime and Position columns
    return dataframe

In [116]:
driver_telemetry = adjust_laptimes(driver_telemetry)

### Min Max and Normalize

In [117]:
from sklearn.preprocessing import MinMaxScaler

In [118]:
scaler = MinMaxScaler()

normalize_columns = ['RPM', 'DistanceToDriverAhead', 'Speed', 'Distance', 'X', 'Y', 'Z', 'RPM', 'Throttle', 'RelativeDistance', 'Laptime']


driver_telemetry[normalize_columns] = scaler.fit_transform(pd.DataFrame(driver_telemetry[normalize_columns]))



In [119]:
driver_telemetry

,DistanceToDriverAhead,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,LapNumber,Laptime
3,0.000036,0.779012,0.000000,1,0.230000,False,1,car,0.000012,0.000012,OnTrack,0.054515,0.308217,0.012532,1.0,0.368108
4,0.000036,0.735428,0.000000,1,0.235344,False,1,pos,0.000000,0.000000,OnTrack,0.054505,0.308208,0.012532,1.0,0.368108
5,0.000036,0.709940,0.000000,1,0.238469,False,1,pos,0.000003,0.000003,OnTrack,0.054505,0.308208,0.012532,1.0,0.368108
6,0.000036,0.697451,0.000000,1,0.240000,False,1,car,0.000012,0.000012,OnTrack,0.054535,0.308235,0.012533,1.0,0.368108
7,0.000015,0.526394,0.023243,1,0.245775,False,1,pos,0.000119,0.000119,OnTrack,0.054900,0.308556,0.012532,1.0,0.368108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,0.999883,0.863612,0.760969,6,1.000000,False,0,pos,0.986435,0.993854,OnTrack,0.034144,0.291796,0.003745,51.0,0.026590
706,0.999906,0.873691,0.767802,6,1.000000,False,0,car,0.987651,0.995079,OnTrack,0.037893,0.294854,0.008415,51.0,0.026590
707,0.999939,0.879664,0.769947,6,1.000000,False,0,pos,0.988796,0.996232,OnTrack,0.042150,0.298349,0.012532,51.0,0.026590
708,1.000000,0.890934,0.773994,6,1.000000,False,0,car,0.990968,0.998421,OnTrack,0.049851,0.304527,0.012537,51.0,0.026590


In [120]:
driver_telemetry['Brake'] = driver_telemetry['Brake'].astype(int)

In [121]:
driver_telemetry

,DistanceToDriverAhead,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,LapNumber,Laptime
3,0.000036,0.779012,0.000000,1,0.230000,0,1,car,0.000012,0.000012,OnTrack,0.054515,0.308217,0.012532,1.0,0.368108
4,0.000036,0.735428,0.000000,1,0.235344,0,1,pos,0.000000,0.000000,OnTrack,0.054505,0.308208,0.012532,1.0,0.368108
5,0.000036,0.709940,0.000000,1,0.238469,0,1,pos,0.000003,0.000003,OnTrack,0.054505,0.308208,0.012532,1.0,0.368108
6,0.000036,0.697451,0.000000,1,0.240000,0,1,car,0.000012,0.000012,OnTrack,0.054535,0.308235,0.012533,1.0,0.368108
7,0.000015,0.526394,0.023243,1,0.245775,0,1,pos,0.000119,0.000119,OnTrack,0.054900,0.308556,0.012532,1.0,0.368108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,0.999883,0.863612,0.760969,6,1.000000,0,0,pos,0.986435,0.993854,OnTrack,0.034144,0.291796,0.003745,51.0,0.026590
706,0.999906,0.873691,0.767802,6,1.000000,0,0,car,0.987651,0.995079,OnTrack,0.037893,0.294854,0.008415,51.0,0.026590
707,0.999939,0.879664,0.769947,6,1.000000,0,0,pos,0.988796,0.996232,OnTrack,0.042150,0.298349,0.012532,51.0,0.026590
708,1.000000,0.890934,0.773994,6,1.000000,0,0,car,0.990968,0.998421,OnTrack,0.049851,0.304527,0.012537,51.0,0.026590


In [122]:
lap_times_label = driver_telemetry.groupby('LapNumber')['Laptime'].max().reset_index()

In [123]:
lap_times_label

,LapNumber,Laptime
0,1.0,0.368108
1,2.0,0.115725
2,3.0,0.089672
3,4.0,0.086136
4,5.0,0.070465
5,6.0,0.061752
6,7.0,0.067070
7,8.0,0.064609
8,9.0,0.066985
9,10.0,0.071002


In [124]:
pad_max = (0,0)
avg = 0
for x in range(1,52):
    curr = driver_telemetry[driver_telemetry['LapNumber'] == x].shape[0]
    
    avg += curr
    if curr > pad_max[1]:
        pad_max = (x, curr)
print("Most telemetry points in 1 lap:", pad_max)
print("Average Telemetry Points per Lap:", avg // 51)

Most telemetry points in 1 lap: (38, 952)
Average Telemetry Points per Lap: 746


In [125]:
driver_telemetry = driver_telemetry.drop(columns=['Laptime'])

In [126]:
padded_data = []
for x in range(1,52):
    lap_data = driver_telemetry[driver_telemetry['LapNumber'] == x]
    telem_points = driver_telemetry[driver_telemetry['LapNumber'] == x].shape[0]
    if telem_points < pad_max[1]:
           padding = pd.DataFrame(np.full((pad_max[1] - telem_points, driver_telemetry.shape[1]), -999), columns= driver_telemetry.columns)
           padding['LapNumber'] = x
           lap_data = pd.concat([driver_telemetry[driver_telemetry['LapNumber'] == x], padding])
    padded_data.append(lap_data)

driver_telemetry = pd.concat(padded_data, ignore_index= True)

In [127]:
driver_telemetry

,DistanceToDriverAhead,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,LapNumber
0,0.000036,0.779012,0.000000,1,0.230000,0,1,car,0.000012,0.000012,OnTrack,0.054515,0.308217,0.012532,1.0
1,0.000036,0.735428,0.000000,1,0.235344,0,1,pos,0.000000,0.000000,OnTrack,0.054505,0.308208,0.012532,1.0
2,0.000036,0.709940,0.000000,1,0.238469,0,1,pos,0.000003,0.000003,OnTrack,0.054505,0.308208,0.012532,1.0
3,0.000036,0.697451,0.000000,1,0.240000,0,1,car,0.000012,0.000012,OnTrack,0.054535,0.308235,0.012533,1.0
4,0.000015,0.526394,0.023243,1,0.245775,0,1,pos,0.000119,0.000119,OnTrack,0.054900,0.308556,0.012532,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48547,-999.000000,-999.000000,-999.000000,-999,-999.000000,-999,-999,-999,-999.000000,-999.000000,-999,-999.000000,-999.000000,-999.000000,51.0
48548,-999.000000,-999.000000,-999.000000,-999,-999.000000,-999,-999,-999,-999.000000,-999.000000,-999,-999.000000,-999.000000,-999.000000,51.0
48549,-999.000000,-999.000000,-999.000000,-999,-999.000000,-999,-999,-999,-999.000000,-999.000000,-999,-999.000000,-999.000000,-999.000000,51.0
48550,-999.000000,-999.000000,-999.000000,-999,-999.000000,-999,-999,-999,-999.000000,-999.000000,-999,-999.000000,-999.000000,-999.000000,51.0


In [128]:
model = keras.Sequential(
    [
        layers.Masking(mask_value= -999.0),
        
        layers.LSTM(64, return_sequences= False),

        layers.Dense(32, activation= 'relu'),
        layers.Dense(1),
    ]
)      

In [129]:
driver_telemetry = driver_telemetry.drop(columns=['Source', 'Status'])

In [130]:
X = []

for lap_num, lap_df in driver_telemetry.groupby("LapNumber"):
    lap_array = lap_df.values  # shape: (952, num_features)
    X.append(lap_array)

X = np.array(X)  # shape: (num_laps, 952, num_features)


y = np.array(lap_times_label['Laptime'])

In [131]:
print(X.shape)
print(y.shape)

(51, 952, 13)
(51,)


In [132]:
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

In [133]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.70588235294, random_state=42)

In [137]:
model.compile(
    optimizer='adam',
    loss='mse',         # Mean Squared Error is standard for regression
    metrics=['mae']     # Tracks Mean Absolute Error during training
)


In [138]:
model.fit(X_train, y_train,
          epochs=20,
          batch_size=8,
          shuffle=True,
          validation_data=(X_test, y_test))

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node sequential_3_1/lstm_3_1/while/TensorListPushBack_24 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\smmey\OneDrive\Documents\f1_lap_by_lap\f1env\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\smmey\OneDrive\Documents\f1_lap_by_lap\f1env\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\smmey\OneDrive\Documents\f1_lap_by_lap\f1env\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\smmey\OneDrive\Documents\f1_lap_by_lap\f1env\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "C:\Users\smmey\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 645, in run_forever

  File "C:\Users\smmey\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1